# Audio Signals

### pull some data from github

In [ ]:
"we import (=clone) all the data or just update (=pull) them"

import os

if not os.path.exists("assets_signal"):
    print("the directory assets_signal is create")
    !git clone https://github.com/vincentvigon/assets_signal
else:
    print("the directory assets_signal is updated")
    %cd assets_signal
    !git pull https://github.com/vincentvigon/assets_signal
    %cd ..


### basic imports

In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import IPython

np.set_printoptions(linewidth=500,precision=3,suppress=True)
plt.style.use("default")

## Sounds

### Import a sound file

I want to import and export sound file with python. How can I do?

I ask to google ->  "python read write sound files"

Often, the best place to see the answer is stackoverflow. I look at the debates:

* Some people say that `PySoundFile` is the more complete lib.
* But I see that `scipy` has also a `io.wavfile` package. Because I already have `scipy`, I try it in first. It is OK to import sounds, but not so good to export them: the generate sounds cannot be easily open.
* so finaly, I used  `PySoundFile`


In [ ]:
!pip install PySoundFile

In [ ]:
"""recall: samplerate is the number of measures make in one second"""
import soundfile as sf
sound, samplerate = sf.read('assets_signal/bornwild.wav')
sound.shape,samplerate

The sound is stereo: it has two columns=two channels: the left one and the right one.

Let's plot this signal. Oscilations are so fast, that we only see their contours. Such plot is called  "waveform" (forme d'onde), and allows to see when the sound is loud or quiet.

In [ ]:
fig,(ax0,ax1)=plt.subplots(2,1,figsize=(8,2),sharex=True)
ax0.plot(sound[:,0])
ax1.plot(sound[:,1])
ax0.set_title("left chanel")
ax1.set_title("right chanel")
fig.tight_layout()

***To you:***

* $(1\heartsuit)$ What is the duration of this sound.
* $(1\heartsuit)$ Add xticks on the above plot which indicate the time.
* $(2\heartsuit)$ Plot the very begining of this signal, to see clearly the oscilations.

In [ ]:
"we plot the half-amplitude-spectrum"
sound0=sound[:,0]
half_spectrum=np.fft.rfft(sound0)
fig,ax=plt.subplots(figsize=(8,2))
N=len(half_spectrum)
freqs=np.linspace(0,samplerate/2,N)
ax.plot(np.abs(half_spectrum)/N);

***To you:*** $(1\heartsuit)$ What is the meaning of the pick at 0 ?

To hear the sound:

In [ ]:
IPython.display.Audio('assets_signal/bornwild.wav')

### Create a sound file

We create an artificial sound. The reference note A (=la) has a frequency  of 440Hz, we make it with a simple sinus-wave.  We add it some harmonics: some weaker sinus-waves wich frequences are $n\times$440Hz.  In the nature, sounds come always with harmonics due to  reasonnance phenomenums.  The dosage of harmonics is different from an instrument to an other, from a voice to another.


Moreover, we produce a sound which is 'crescendo'  (more and more loud).

***To you:*** $(1\heartsuit)$ Create a decrecendo (=diminuendo) soud.



In [ ]:
samplerate=44100
duration=3 #secondes
t=np.linspace(0,duration,duration*samplerate)

harmonics=[2,3,5]
hamonics_intensity=[1/2,1/4,1/3]

signal =  np.sin(2 * np.pi * 440 * t)
"""ajout des harmoniques"""
for h,h_i in zip(harmonics,hamonics_intensity):
    signal+= h_i*np.sin(h* 2 * np.pi * 440 * t )

In [ ]:
fig,ax=plt.subplots(figsize=(8,1))
ax.plot(t[:200],signal[:200]);

In [ ]:
"""we must be carful: usualy, intensity of sound must stay in [0,1]"""
maxiSig=np.max(signal)
volume=t/t[-1]/maxiSig/2
signal*=volume

fig,ax=plt.subplots(figsize=(8,1))
ax.plot(t,signal);

In [ ]:
sf.write('la440.wav', signal, samplerate)

In [ ]:
IPython.display.Audio('la440.wav')

### Create a music

To create a music, we have to concatenate several note. But  transitions must be smooth to avoid some short noise 'tack'.  Observe:

In [ ]:
samplerate = 11025

duration0=1.3
duration1=2.6

nb0=int(duration0*samplerate)
nb1=int(duration1*samplerate)
nb=nb0+nb1
t=np.linspace(0,duration0+duration1,nb)
signal0=np.sin(2*np.pi*t*2)
signal1=np.sin(2*np.pi*t*5)

window0 = np.zeros(nb)
window1 = np.zeros(nb)


In [ ]:
"""transition abrupte"""
window0[:nb0]=1
window1[nb0:]=1

In [ ]:
def plot_all():
    plt.subplot(3,1,1)
    plt.plot(window0)
    plt.plot(window0*signal0)
    plt.ylim([-2,2])

    plt.subplot(3,1,2)
    plt.plot(window1)
    plt.plot(window1*signal1)
    plt.ylim([-2, 2])

    plt.subplot(3,1,3)
    plt.plot(window0+window1)
    plt.plot(window0*signal0+window1*signal1)
    plt.ylim([-2, 2])

plot_all()

In [ ]:
sf.write("raw_transi.wav",window0*signal0+window1*signal1,samplerate)

In [ ]:
IPython.display.Audio("raw_transi.wav")

The previous sound is not audible (too low frequency), but you can hear the raw transition.

In [ ]:
demiTransition = int(0.1 * samplerate)
montee = np.linspace(0, 1, 2 * demiTransition)
descente = 1 - montee
window0[:nb0 - demiTransition] = 1
window0[nb0 - demiTransition:nb0 + demiTransition] = descente
window1[nb0 + demiTransition:] = 1
window1[nb0 - demiTransition:nb0 + demiTransition] = montee

In [ ]:
plot_all()

In [ ]:
sf.write("soft_transi.wav",window0*signal0+window1*signal1,samplerate)

In [ ]:
IPython.display.Audio("soft_transi.wav")

Now the transition is soft, we get a perfect silence.

***To you:*** $(2\heartsuit)$ Remake these two transitions, but with audible sounds.

***Bonus:*** Write a little melody. The correspondance "note $\to$ frequency" can be easily found on the net. To win:

* $4\star$ for a five notes melody with smooth transition of course.
* $4\star$ more, if we recognise a true musical melody  
* $4\star$ more if you optimize the previous code:  I have created one long vector per note: instead, you can create only one long vector and change it part by part to make the melody, or create several short vectors and concatenate them (but this second solution is only for sharp transition).  

Structure de l'algo
        #transition brutale
        np.concatenate(notes)
        #ou notes est une liste de petit signaux

        #transition douce
        signal_total=np.zeros()
        for note in notes
            signal_total[deb,fin]+=note

### Spectrogram

To analyse long signal, it is often better to make a spectrogram than a simple Fourier Transform. Here is the process:

* We cut the signal in short slices, sufficiently short so that, in every slice, the signal is homogenious (like a simple mixture of sinus-waves).
* Then we compute the fft for every slice.
*  fft are stack as in columns of a matrix
* This matrix is plot with a colormap.

Such process is also called a "time-frequency analysis"

In [ ]:
import scipy.signal

epsilon = 0.0001
t = np.arange(0, 1, epsilon)

sig_debut = 0.5 * np.sin(t * 2 * np.pi * 440) + np.sin(t * 2 * np.pi * 220)
sig_fin = 0.5 * np.sin(t * 2 * np.pi * 880) + np.sin(t * 2 * np.pi * 440)
sig=np.concatenate((sig_debut, sig_fin))


"""from the official help : https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html """
f, t, Sxx = scipy.signal.spectrogram(sig, 1/epsilon)
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

***To you:*** $(3\heartsuit)$ Plot the spectrogram of some  short sounds of your choice (put it before in your working directory). Try to link what you hear and what you see.

In [ ]:
#example with born to be wilde
f, t, Sxx = scipy.signal.spectrogram(sound[:,0], 1/epsilon)
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

***A vous:*** Où voit-on que le chanteur utilise des "effets de voix"?

### Exo: Sound filtering

In [ ]:
sound, samplerate = sf.read('assets_signal/sound_surprise.wav')
sound.shape,samplerate

In [ ]:
IPython.display.Audio('assets_signal/sound_surprise.wav')

***To you:***
* What is the duration of the signal. Justify!
* Plot the whole sound, with the good scale of time
* Plot the very begining of this sound, to see the oscillations.
* Plot the half-amplitude-spectrum with the good scale of frequencies
* What is the musical name of this sound. How many "harmonics"?
* With the help of fft, suppress all harmonics to keep only a "pure"
sound.  
* Transform the initial sound to make a "crescendo" effect (= more and more loud)
* Plot the spectrogram of the transformed sound